# Обучение модели машинного перевода с русского на алеутский язык

Этот ноутбук содержит код для обучения модели машинного перевода с русского на алеутский язык. Модель основана на архитектуре MarianMT из библиотеки `transformers` и обучается на пользовательском наборе данных `russian_aleut_dataset.csv`, содержащем пары переводов.

## Требования
- Python 3.11 (рекомендуется).
- Google Colab с доступом к GPU (для ускорения обучения).
- Датасет `russian_aleut_dataset.csv` (должен быть загружен в Colab).

## Зависимости
Установим необходимые библиотеки:

In [ ]:
!pip install transformers torch pandas datasets

## Импорт библиотек и настройка устройства

Импортируем необходимые библиотеки и определяем устройство (GPU или CPU) для обучения.

In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from google.colab import files
from transformers import (
    MarianMTModel,
    MarianTokenizer,
    Trainer,
    TrainingArguments,
)

# Определяем устройство для обучения (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

## Загрузка и подготовка данных

Загружаем датасет из файла `russian_aleut_dataset.csv` и преобразуем его в формат `Dataset` для работы с библиотекой `transformers`.

In [ ]:
def load_dataset(file_path: str) -> Dataset:
    """Загружает датасет из CSV-файла и преобразует его в формат Dataset.

    Args:
        file_path: Путь к CSV-файлу с данными (русский-алеутский).

    Returns:
        Dataset: Набор данных в формате Hugging Face Dataset.
    """
    # Читаем CSV-файл с парами переводов
    df = pd.read_csv(file_path, sep=";", encoding="utf-8")
    translation_pairs = df[["Russian", "Aleut"]].dropna().to_dict("records")

    # Преобразуем в формат Dataset
    dataset = Dataset.from_list(
        [{"source": pair["Russian"], "target": pair["Aleut"]} for pair in translation_pairs]
    )
    return dataset

# Загружаем датасет
dataset = load_dataset("russian_aleut_dataset.csv")

## Инициализация модели и токенизатора

Инициализируем предобученную модель MarianMT и токенизатор, добавляем спецсимвол `ẍ`, характерный для алеутского языка.

In [ ]:
def initialize_model_and_tokenizer(model_name: str) -> tuple[MarianMTModel, MarianTokenizer]:
    """Инициализирует модель и токенизатор MarianMT."""
    
    # Загружаем токенизатор и модель
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name).to(device)

    # Добавляем спецсимвол "ẍ" в токенизатор и обновляем размер эмбеддингов
    tokenizer.add_tokens(["ẍ"])
    model.resize_token_embeddings(len(tokenizer))

    # Проверяем токенизацию и декодирование на тестовом тексте
    sample_text = "qana-ẍ angali-ẍ ula-ẍ a-ku-ẍ"
    encoded = tokenizer(sample_text, return_tensors="pt")
    decoded = tokenizer.decode(encoded["input_ids"][0], skip_special_tokens=True)
    print("Токенизировано и декодировано:", decoded)

    return model, tokenizer

# Инициализируем модель и токенизатор
model_name = "Helsinki-NLP/opus-mt-ru-en"
model, tokenizer = initialize_model_and_tokenizer(model_name)

## Предобработка данных

Токенизируем входные (русские) и целевые (алеутские) тексты, подготавливая их для обучения.

In [ ]:
def preprocess_function(examples: dict) -> dict:
    """Подготавливает данные для обучения: токенизирует входные и целевые тексты."""
    
    inputs = examples["source"]
    targets = examples["target"]

    # Токенизируем входные тексты (русский)
    model_inputs = tokenizer(
        inputs,
        max_length=128,
        truncation=True,
        padding="max_length",
    )

    # Токенизируем целевые тексты (алеутский) в режиме целевого токенизатора
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=128,
            truncation=True,
            padding="max_length",
        )

    # Добавляем метки (labels) в выходной словарь
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Применяем предобработку к датасету
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Разделяем на обучающую и тестовую выборки (90% — обучение, 10% — тест)
train_test = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test["train"]
eval_dataset = train_test["test"]

## Обучение модели

Настраиваем параметры обучения и запускаем процесс дообучения модели.

In [ ]:
def train_model(
    model: MarianMTModel,
    tokenizer: MarianTokenizer,
    train_dataset: Dataset,
    eval_dataset: Dataset,
) -> None:
    """Обучает модель на подготовленных данных."""
    
    # Настраиваем параметры обучения
    training_args = TrainingArguments(
        output_dir="./mbart_aleut",
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=5,
        weight_decay=0.01,
        save_strategy="epoch",
        load_best_model_at_end=True,
        logging_dir="./logs",
        logging_steps=10,
        report_to="none",
        fp16=True,
    )

    # Инициализируем Trainer для обучения
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )

    # Запускаем обучение
    trainer.train()

# Обучаем модель
train_model(model, tokenizer, train_dataset, eval_dataset)

## Сохранение модели

Сохраняем обученную модель и токенизатор в папку `Marian_aleut_model`, архивируем её и скачиваем архив.

In [ ]:
def save_model_and_tokenizer(
    model: MarianMTModel,
    tokenizer: MarianTokenizer,
    save_dir: str,
    zip_name: str,
) -> None:
    """Сохраняет модель и токенизатор, а затем архивирует их в zip-файл."""
    
    # Сохраняем модель и токенизатор
    model.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)

    # Архивируем папку с моделью
    !zip -r {zip_name} {save_dir}

    # Скачиваем архив
    files.download(zip_name)

# Сохраняем модель
save_model_and_tokenizer(model, tokenizer, "Marian_aleut_model", "Marian_aleut_model.zip")

## Тестирование модели

Проверяем работу модели на нескольких тестовых фразах.

In [ ]:
def translate(text: str, model: MarianMTModel, tokenizer: MarianTokenizer) -> str:
    """Переводит текст с русского на алеутский с использованием обученной модели."""
    
    # Токенизируем входной текст
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)

    # Генерируем перевод
    translated_tokens = model.generate(
        **inputs,
        max_length=128,
        num_beams=5,
        early_stopping=True,
    )

    # Декодируем результат
    result = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    # Исправляем пробелы перед спецсимволом "ẍ"
    result = result.replace(" ẍ", "ẍ")
    return result

# Тестируем модель
print("Тестирование модели:")
test_phrases = ["Где большой дом?", "Кто видит реку?"]
for phrase in test_phrases:
    translation = translate(phrase, model, tokenizer)
    print(f"Русский: {phrase} -> Алеутский: {translation}")